In [1]:
#dependencies and setup
import pandas as pd
import os
from openpyxl import Workbook
import numpy as np
import requests
import json
import re
from pprint import pprint
from bs4 import BeautifulSoup
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import time

In [2]:
# SQLite dependencies
import sqlite3
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Float

# Extraction

## School performance score

In [ ]:
# create a browser instance using splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
time.sleep(1)

In [ ]:
# visit the NJ schools url
nj_school_rate_URL = 'https://rc.doe.state.nj.us/ReportsDatabase.aspx'
browser.visit(nj_school_rate_URL)
time.sleep(1)

# create HTML object
html = browser.html

# parse HTML with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
# access html elements to get the link for the table data
div = soup.find(id= 'ContentPlaceHolder1_LatestDataFilesPanel')
download_file_link_partial = div.find('a', href = True)['href']
download_file_link_partial

In [ ]:
# creating the full url for the latest file
base_url = "https://rc.doe.state.nj.us/"
download_file_link_partial = 'ReportsDatabase/PerformanceReports.xlsx'
download_file_link = base_url + download_file_link_partial
download_file_link

In [ ]:
#  define data types for a couple of columns
info_dtype_dic= {'COUNTY_CODE' : str, 'DISTRICT_CODE':str, 'SCHOOL_CODE':str}

In [ ]:
#  import school metadata
school_info = pd.read_excel(download_file_link,engine='openpyxl',sheet_name="Header and Contact", dtype = info_dtype_dic)
school_info = school_info[["COUNTY_CODE", "COUNTY_NAME", "DISTRICT_CODE", "DISTRICT_NAME", "SCHOOL_CODE", "SCHOOL_NAME", "GRADESPAN"]]
school_info.columns= school_info.columns.str.lower()
school_info['county_name'] = school_info['county_name'].str.upper()
school_info.head()

In [ ]:
#  define data types for a couple of columns
growth_dtype_dic= {'CountyCode' : str, 'DistrictCode':str, 'SchoolCode':str}

In [ ]:
#  import school student growth data
school_growth_src = pd.read_excel(download_file_link,engine='openpyxl',sheet_name="StudentGrowth", dtype = growth_dtype_dic)
school_growth_src.head()

In [ ]:
#  fixing data types for numeric columns
school_growth_src['SchoolMedian'] = school_growth_src['SchoolMedian'].apply(pd.to_numeric, errors='coerce')
school_growth_src['DistrictMedian'] = school_growth_src['DistrictMedian'].apply(pd.to_numeric, errors='coerce')
school_growth_src.head()

In [ ]:
#  filtering out bad data and replaceing null scores with district average
school_growth = school_growth_src[(school_growth_src['StudentGroup'] == "Schoolwide") & (school_growth_src['DistrictMedian'].notnull())]
school_growth["SchoolMedian"].fillna(school_growth["DistrictMedian"], inplace=True)
# school_growth.loc[(school_growth['SchoolMedian'] == 0) & (school_growth["SchoolMedian"].isnull()), 'SchoolMedian'] = school_growth["DistrictMedian"]

school_growth.head()

In [ ]:
#  aggreagating the dataframe to get the ranking per school
school_growth.rename(columns = {"CountyCode": 'county_code', "CountyName": "county_name",
                               "DistrictCode": "district_code", "DistrictName": "district_name",
                               "SchoolCode": "school_code", "SchoolName": "school_name"}, inplace = True)

school_growth = school_growth.groupby(["county_code", "county_name", "district_code", "district_name", "school_code", "school_name"]\
                                      , as_index =False).agg(rating = ("SchoolMedian", "mean"))
school_growth['county_name'] = school_growth['county_name'].str.upper()
school_growth.head()

In [ ]:
#  Check for nulls
school_growth[school_growth["rating"].isnull()]

In [ ]:
# joining the 2 dataframes to create the final dataset for NJ school 
school_rating_df = pd.merge(school_info, school_growth, how = "inner", on = ['county_code', 'county_name', 'district_code', 'district_name', 'school_code', 'school_name'])
school_rating_df.head()

In [ ]:
#  Check for nulls
school_rating_df[school_rating_df["rating"].isnull()]

In [ ]:
# close the browser session    
browser.quit()

## Poverty 2019 (latest)

In [ ]:
# URL for county poverty data
poverty_url = 'https://api.census.gov/data/timeseries/poverty/saipe?get=NAME,SAEMHI_PT,SAEMHI_MOE,SAEPOVALL_PT,SAEPOVALL_MOE,SAEPOVRTALL_MOE,SAEPOVRTALL_PT,STABREV&for=county:*&in=state:34&YEAR=2019'

In [ ]:
# API call to get countries metadata
poverty_data = requests.get(poverty_url).json()
pprint(poverty_data)

In [ ]:
# initialize empty lists
county_name = []
median_hh_income = []
median_hh_inc_moe = []
poverty_count = []
poverty_count_moe = []
poverty_rate = []
poverty_rate_moe = []
st_abb =[]
state_fips = []
county_fips = []

In [ ]:
# iterate through the response to populate the lists and create a dataframe
for i in range(1,len(poverty_data)):
    county_name.append(poverty_data[i][0])
    median_hh_income.append(poverty_data[i][1])
    median_hh_inc_moe.append(poverty_data[i][2])
    poverty_count.append(poverty_data[i][3])
    poverty_count_moe.append(poverty_data[i][4])
    poverty_rate.append(poverty_data[i][5])
    poverty_rate_moe.append(poverty_data[i][6])
    st_abb.append(poverty_data[i][7])
    state_fips.append(poverty_data[i][9])
    county_fips.append(poverty_data[i][10])
counties_poverty_df = pd.DataFrame({"county_name": county_name,
                                    "median_hh_income": median_hh_income,
                                    "median_hh_inc_moe": median_hh_inc_moe,
                                    "poverty_count": poverty_count,
                                    "poverty_count_moe": poverty_count_moe,
                                    "poverty_rate": poverty_rate,
                                    "poverty_rate_moe": poverty_rate_moe,
                                    "st_abb": st_abb,
                                    "state_fips": state_fips,
                                    "county_fips": county_fips})
counties_poverty_df.head()

In [ ]:
# Manipulating the column name to match across all the datasets
counties_poverty_df['county_name']= counties_poverty_df.county_name.str.replace("County","")
counties_poverty_df['county_name']= counties_poverty_df['county_name'].str.upper()
counties_poverty_df['county_name']= counties_poverty_df['county_name'].str.strip()
counties_poverty_df

In [ ]:
# unique county count
counties_poverty_df['county_name'].nunique()

## NJ Tax 2020

In [ ]:
# create a browser instance using splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
time.sleep(1)

In [ ]:
# visit the Mars News url
crime_NJ_URL = 'https://www.state.nj.us/treasury/taxation/lpt/taxrate.shtml'
browser.visit(crime_NJ_URL)
time.sleep(2)

# create HTML object
html = browser.html

# parse HTML with BeautifulSoup
soup = BeautifulSoup(html, 'lxml')

In [ ]:
div = soup.find('div', class_ = 'col-lg-10 col-xl-10')
download_file_link_partial = div.find_all('a', href = True, target =True)[0]['href']
download_file_link_partial

In [ ]:
base_url = "https://www.state.nj.us/treasury/taxation/lpt/"
download_file_link = base_url + download_file_link_partial
download_file_link

In [ ]:
xls = pd.ExcelFile(download_file_link)
tax_sheet = xls.sheet_names
print(tax_sheet)

In [ ]:
dtype_dic= {'Cty/District Code' : str, 'CNTY/DIST CODE':str}

In [ ]:
NJ_gen_tax_df = pd.read_excel(download_file_link,engine='xlrd',sheet_name=tax_sheet[0], skiprows=range(1), dtype = dtype_dic).dropna(how= "any")
NJ_gen_tax_df = NJ_gen_tax_df.iloc[:, [0,1,2,len(NJ_gen_tax_df.columns)-1]]
NJ_gen_tax_df.iloc[:,1] = NJ_gen_tax_df.iloc[:,1].apply(lambda x: x.zfill(4))
NJ_gen_tax_df.rename(columns = {'County': 'county_name', 'Cty/District Code':'district_code', 'District':'district_name', 'GENERAL.23': 'general_tax_rate'},inplace = True)
NJ_gen_tax_df.reset_index(drop= True, inplace = True)
NJ_gen_tax_df.head()

In [ ]:
NJ_eff_tax_df = pd.read_excel(download_file_link,engine='xlrd',sheet_name=tax_sheet[1], skiprows=range(1), dtype = dtype_dic).dropna(how= "any")
NJ_eff_tax_df = NJ_eff_tax_df.iloc[:, [0,1,2,len(NJ_eff_tax_df.columns)-1]]
NJ_eff_tax_df.iloc[:,1] = NJ_eff_tax_df.iloc[:,1].apply(lambda x: x.zfill(4))
NJ_eff_tax_df.rename(columns = {'COUNTY': 'county_name', 'CNTY/DIST CODE':'district_code', 'DISTRICT':'district_name', 'EFFECTIVE.23': 'effective_tax_rate'},inplace = True)
NJ_eff_tax_df.reset_index(drop= True, inplace = True)
NJ_eff_tax_df.head()

In [ ]:
# close the browser session    
browser.quit()

In [ ]:
NJ_tax_df = pd.merge(NJ_gen_tax_df, NJ_eff_tax_df, how = 'left', on = ['county_name', 'district_code', 'district_name'])
NJ_tax_df.head()

## County crime reports 2020

In [ ]:
# create a browser instance using splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
time.sleep(1)

In [ ]:
# visit the Mars News url
crime_NJ_URL = 'https://www.njsp.org/ucr/current-crime-data.shtml'
browser.visit(crime_NJ_URL)
time.sleep(2)

# create HTML object
html = browser.html

# parse HTML with BeautifulSoup
soup = BeautifulSoup(html, 'lxml')

In [ ]:
# use splinter to click on the 'Continue' button to retrieve a full-size jpg url
browser.find_by_tag("input").last.click()
time.sleep(1)

In [ ]:
# create HTML object
html = browser.html
# parse HTML with BeautifulSoup
soup = BeautifulSoup(html, 'lxml')
# fetch the table
table = soup.find('table')
# fetch the partial link for donwloadable file
download_item = table.find_all('a', href = True)
download_link= download_item[0]['href']
# create full download link
base_url= 'https://www.njsp.org/ucr/'
full_file_link = base_url+download_link
print(full_file_link)

In [ ]:
# close the browser session    
browser.quit()

In [ ]:
# create xlsx object with the file from the link
xls = pd.ExcelFile(full_file_link,
     engine='openpyxl')
# get all the worksheet names
crime_sheet = xls.sheet_names
print(crime_sheet)

In [ ]:
# unique county
len(crime_sheet[1:])

In [ ]:
# create separate dataframes for each worksheet
for i in range(1,len(crime_sheet)):
    globals()['crime_%s' % crime_sheet[i].lower().replace(" ", "_")] = pd.read_excel(full_file_link,
     engine='openpyxl',sheet_name=crime_sheet[i], skiprows=range(3)).dropna(how="all")
    globals()['crime_%s' % crime_sheet[i].lower().replace(" ", "_")]["county_name"] = '%s' % crime_sheet[i].upper()
    globals()['crime_%s' % crime_sheet[i].lower().replace(" ", "_")].drop(globals()['crime_%s' % crime_sheet[i].lower().replace(" ", "_")].tail(7).index, inplace = True)
    print('crime_%s' % crime_sheet[i].lower().replace(" ", "_"))

### Offline copy import

In [ ]:
# del [crime_atlantic,
# crime_bergen,
# crime_burlington,
# crime_camden,
# crime_cape_may,
# crime_cumberland,
# crime_essex,
# crime_gloucester,
# crime_hudson,
# crime_hunterdon,
# crime_mercer,
# crime_middlesex,
# crime_monmouth,
# crime_morris,
# crime_ocean,
# crime_passaic,
# crime_salem,
# crime_somerset,
# crime_sussex,
# crime_union,
# crime_warren]

In [ ]:
# source : https://www.njsp.org/ucr/current-crime-data1.shtml?agree=0
# xls = pd.ExcelFile(os.path.join("data", "2020_UCR_Jan-Sept_v2.xlsx"),
#      engine='openpyxl')

In [ ]:
# crime_sheet = xls.sheet_names
# crime_sheet

In [ ]:
# for i in range(1,len(crime_sheet)):
#     globals()['crime_%s' % crime_sheet[i].lower()] = pd.read_excel(os.path.join("data", "2020_UCR_Jan-Sept_v2.xlsx"),
#      engine='openpyxl',sheet_name=crime_sheet[i], skiprows=range(3)).dropna(how="all")
#     print('crime_%s' % crime_sheet[i].lower())

In [ ]:
# concat all the dataframes
all_crime_df = pd.concat([crime_atlantic,
crime_bergen,
crime_burlington,
crime_camden,
crime_cape_may,
crime_cumberland,
crime_essex,
crime_gloucester,
crime_hudson,
crime_hunterdon,
crime_mercer,
crime_middlesex,
crime_monmouth,
crime_morris,
crime_ocean,
crime_passaic,
crime_salem,
crime_somerset,
crime_sussex,
crime_union,
crime_warren], ignore_index = True)

In [ ]:
# make all the column name lower case
all_crime_df.columns= all_crime_df.columns.str.lower()
all_crime_df.columns= all_crime_df.columns.str.replace(" ","_")
all_crime_df.head(10)

### Population dataset

In [ ]:
population_df = all_crime_df[["county_name","agency", "orinumber", "population"]].dropna(how="any")
population_df["population"] = population_df["population"].astype(int)
population_df.drop_duplicates(keep='first', inplace= True, ignore_index=True)
population_df.head(-10)

In [ ]:
# unique county count
population_df['county_name'].nunique()

In [ ]:
len(population_df)

### Crime detail

In [ ]:
# create a list of all police department
pol_dep = []
for i in range(len(population_df["agency"])):
    for k in range(5):
        pol_dep.append(population_df["agency"][i])
print(pol_dep)

In [ ]:
len(pol_dep)

In [ ]:
# Alogning the crime data to ingest the police deprtment list as a column
crime_det_df = all_crime_df[["county_name","agency", "murder", "rape", "robbery", "assault", "burglary",\
                         "larceny", "auto_theft", "total"]].dropna(how="any")

crime_det_df.rename(columns = {"agency": "report_type"}, inplace =True)

crime_det_df = crime_det_df.loc[crime_det_df["report_type"] != crime_det_df["report_type"].shift()]
crime_det_df.reset_index(inplace= True, drop = True)
crime_det_df["police_dept"] = pol_dep
crime_det_df = crime_det_df[["county_name","police_dept", "report_type", "murder", "rape", "robbery", "assault", "burglary",\
                         "larceny", "auto_theft", "total"]]
crime_det_df.head(-1)

### Crime summary dataset

In [ ]:
# creatuion of crime summary dataset
crime_df = all_crime_df[["county_name","agency", "murder", "rape", "robbery", "assault", "burglary",\
                         "larceny", "auto_theft", "total"]].dropna(how="any")
crime_df.reset_index(inplace= True, drop = True)
crime_df.rename(columns = {"agency": "report_type"}, inplace =True)
crime_df = crime_df.groupby(['county_name', 'report_type'],as_index=False).agg(murder = ('murder', 'sum'), rape = ('rape', 'sum'),\
                                                               robbery = ('robbery', 'sum'),assault = ('assault', 'sum'),\
                                                               burglary = ('burglary', 'sum'),larceny = ('larceny', 'sum'),\
                                                               auto_theft = ('auto_theft', 'sum'),total = ('total', 'sum'))
crime_df.head(-10)

In [ ]:
# unique county count
crime_df['county_name'].nunique()

In [ ]:
# county record count
crime_df['county_name'].value_counts()

# DB load

In [ ]:
# Connect to the NJ_CPS database
connection  = sqlite3.connect("Resources/NJ_CPS.sqlite")

# Get a cursor object
cursor = connection.cursor()

# Execute the DROP Table SQL statement
drop_school = "DROP TABLE IF EXISTS NJ_school_rating"

cursor.execute(drop_school)

drop_poverty = "DROP TABLE IF EXISTS NJ_poverty"

cursor.execute(drop_poverty)

drop_population = "DROP TABLE IF EXISTS NJ_population"

cursor.execute(drop_population)

drop_crime = "DROP TABLE IF EXISTS NJ_crime"

cursor.execute(drop_crime)

drop_crime_det = "DROP TABLE IF EXISTS NJ_crime_detail"

cursor.execute(drop_crime_det)

drop_tax = "DROP TABLE IF EXISTS NJ_tax"

cursor.execute(drop_tax)

drop_geojson = "DROP TABLE IF EXISTS NJ_geojson"

cursor.execute(drop_geojson)

drop_house = "DROP TABLE IF EXISTS NJ_house_price"

cursor.execute(drop_house)

In [4]:
#  SQLite DB creation and establishing connection
database_path = "../Resources/NJ_CPS.sqlite"
engine = create_engine(f"sqlite:///{database_path}", echo=True)
sqlite_connection = engine.connect()

2021-04-18 16:53:46,045 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-04-18 16:53:46,047 INFO sqlalchemy.engine.base.Engine ()
2021-04-18 16:53:46,048 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-04-18 16:53:46,048 INFO sqlalchemy.engine.base.Engine ()


# Loading zillow data in to DB

In [5]:
zillow_df = pd.read_csv('../Resources/zillow_house_cleaned.csv')
zillow_df.head()

,county,year,price
0,Atlantic,1996,108995.08
1,Atlantic,1997,111600.33
2,Atlantic,1998,115382.33
3,Atlantic,1999,120679.00
4,Atlantic,2000,128581.33


In [10]:
sqlite_table = "NJ_house_price"
zillow_df.to_sql(sqlite_table, sqlite_connection, if_exists='replace', index=False)

2021-04-18 17:05:40,454 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("NJ_house_price")
2021-04-18 17:05:40,455 INFO sqlalchemy.engine.base.Engine ()
2021-04-18 17:05:40,457 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("NJ_house_price")
2021-04-18 17:05:40,458 INFO sqlalchemy.engine.base.Engine ()
2021-04-18 17:05:40,460 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "NJ_house_price" (
	county TEXT, 
	year BIGINT, 
	price FLOAT
)


2021-04-18 17:05:40,461 INFO sqlalchemy.engine.base.Engine ()
2021-04-18 17:05:40,475 INFO sqlalchemy.engine.base.Engine COMMIT
2021-04-18 17:05:40,477 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-04-18 17:05:40,480 INFO sqlalchemy.engine.base.Engine INSERT INTO "NJ_house_price" (county, year, price) VALUES (?, ?, ?)
2021-04-18 17:05:40,481 INFO sqlalchemy.engine.base.Engine (('Atlantic', 1996, 108995.08), ('Atlantic', 1997, 111600.33), ('Atlantic', 1998, 115382.33), ('Atlantic', 1999, 120679.0), ('Atlantic', 2000, 1285

## Loading county tax in to DB

In [ ]:
sqlite_table = "NJ_tax"
NJ_tax_df.to_sql(sqlite_table, sqlite_connection, if_exists='replace', index=False)

## Loading school ratings in to DB

In [ ]:
sqlite_table = "NJ_school_rating"
school_rating_df.to_sql(sqlite_table, sqlite_connection, if_exists='replace', index=False)

## Loading poverty rates in DB

In [ ]:
sqlite_table = "NJ_poverty"
counties_poverty_df.to_sql(sqlite_table, sqlite_connection, if_exists='replace', index=False)

## Loading crime detail data in DB

In [ ]:
sqlite_table = "NJ_crime_detail"
crime_det_df.to_sql(sqlite_table, sqlite_connection, if_exists='replace', index=False)

## Loading crime data in DB

In [ ]:
sqlite_table = "NJ_crime"
crime_df.to_sql(sqlite_table, sqlite_connection, if_exists='replace', index=False)

## Loading population data in DB

In [ ]:
sqlite_table = "NJ_population"
population_df.to_sql(sqlite_table, sqlite_connection, if_exists='replace', index=False)

# DB query Test

In [14]:
query = "SELECT * FROM NJ_poverty"
df = pd.read_sql(query, sqlite_connection)
df.head()

2021-03-20 10:53:11,046 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("SELECT * FROM NJ_poverty")
2021-03-20 10:53:11,047 INFO sqlalchemy.engine.base.Engine ()
2021-03-20 10:53:11,048 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("SELECT * FROM NJ_poverty")
2021-03-20 10:53:11,049 INFO sqlalchemy.engine.base.Engine ()
2021-03-20 10:53:11,049 INFO sqlalchemy.engine.base.Engine SELECT * FROM NJ_poverty
2021-03-20 10:53:11,050 INFO sqlalchemy.engine.base.Engine ()


,county_name,median_hh_income,median_hh_inc_moe,poverty_count,poverty_count_moe,poverty_rate,poverty_rate_moe,st_abb,state_fips,county_fips
0,ATLANTIC,62678,2822,29057,4251,1.6,11.3,NJ,34,001
1,BERGEN,107971,3025,52980,7662,0.8,5.7,NJ,34,003
2,BURLINGTON,88443,3233,24961,4374,1,5.7,NJ,34,005
3,CAMDEN,73168,2374,53641,7048,1.4,10.7,NJ,34,007
4,CAPE MAY,66565,4753,8853,1981,2.2,9.8,NJ,34,009


In [15]:
df.to_csv("data/poverty.csv", index = False)

In [ ]:
metadata_df = pd.read_sql_query("SELECT T1.*, T2.population, T3.general_tax_rate, T3.effective_tax_rate FROM (SELECT county_name,median_hh_income,poverty_rate FROM NJ_poverty) AS T1 \
                                INNER JOIN (SELECT county_name, SUM(population) as population FROM NJ_population GROUP BY county_name) AS T2\
                                ON T1.county_name = T2.county_name \
                                INNER JOIN (SELECT county_name, AVG(general_tax_rate) as general_tax_rate, AVG(effective_tax_rate) as effective_tax_rate FROM NJ_tax GROUP BY county_name) AS T3 \
                                ON T1.county_name = T3.county_name", sqlite_connection)
metadata_df["median_hh_income"] = metadata_df["median_hh_income"].astype(int)
metadata_df

In [ ]:
crime_df = pd.read_sql_query("SELECT county_name, murder,rape, robbery, assault, burglary, larceny, auto_theft, total FROM NJ_crime WHERE report_type = 'Number of Offenses'", sqlite_connection)
crime_df.head()

In [ ]:
query = '''SELECT POV.county_name, POV.median_hh_income, POV.poverty_rate, POV.county_fips,
           CRM.total AS total_offense, ROUND(CRM2.total,2) AS rate_per_100k, CRM3.total AS total_arrest,
           POP.population, SCH.school_rank, (TAX.general_tax_rate + TAX.effective_tax_rate)/2 AS tax_rate
           FROM NJ_poverty AS POV 
           INNER JOIN NJ_crime AS CRM ON POV.county_name = CRM.county_name AND CRM.report_type = 'Number of Offenses'
           INNER JOIN NJ_crime AS CRM2 ON POV.county_name = CRM2.county_name AND CRM2.report_type = 'Rate Per 100,000'
           INNER JOIN NJ_crime AS CRM3 ON POV.county_name = CRM3.county_name AND CRM3.report_type = 'Number of Arrests'
           INNER JOIN (SELECT county_name, SUM(population) AS population FROM NJ_population GROUP BY county_name) AS POP ON POV.county_name = POP.county_name
           INNER JOIN (SELECT county_name, ROUND(AVG(summativescore),2) AS school_rank FROM NJ_school_rating GROUP BY county_name) AS SCH ON POV.county_name = SCH.county_name
           INNER JOIN (SELECT county_name, AVG(general_tax_rate) as general_tax_rate, AVG(effective_tax_rate) as effective_tax_rate FROM NJ_tax GROUP BY county_name) AS TAX ON POV.county_name = TAX.county_name'''
df = pd.read_sql(query, sqlite_connection)
df.head()

In [ ]:
query2 = '''SELECT DISTINCT county_name||"-"||police_dept AS id ,police_dept AS label,  "NJ-"||county_name AS parent, SUM(count) AS value FROM 
        (SELECT county_name,police_dept, 'murder' AS crime_type, SUM(murder) AS count FROM NJ_crime WHERE report_type = 'Number of Offenses' GROUP BY 1,2,3
    UNION ALL SELECT county_name, police_dept, 'rape' AS crime_type, SUM(rape) AS count FROM NJ_crime WHERE report_type = 'Number of Offenses' GROUP BY 1,2,3
    UNION ALL SELECT county_name, police_dept, 'robbery' AS crime_type, SUM(robbery) AS count FROM NJ_crime WHERE report_type = 'Number of Offenses' GROUP BY 1,2,3
    UNION ALL SELECT county_name, police_dept,'assault' AS crime_type, SUM(assault) AS count FROM NJ_crime WHERE report_type = 'Number of Offenses' GROUP BY 1,2,3
    UNION ALL SELECT county_name, police_dept,'burglary' AS crime_type, SUM(burglary) AS count FROM NJ_crime WHERE report_type = 'Number of Offenses' GROUP BY 1,2,3
    UNION ALL SELECT county_name, police_dept,'larceny' AS crime_type, SUM(larceny) AS count FROM NJ_crime WHERE report_type = 'Number of Offenses' GROUP BY 1,2,3
    UNION ALL SELECT county_name, police_dept,'auto_theft' AS crime_type, SUM(auto_theft) AS count FROM NJ_crime WHERE report_type = 'Number of Offenses' GROUP BY 1,2,3)
    GROUP BY 1,2,3'''
df2 = pd.read_sql(query2, sqlite_connection)
df2.head()

In [ ]:
query = '''SELECT POV.county_name, POV.median_hh_income, POV.poverty_rate, POV.county_fips,
               CRM.total AS total_offense, CRM2.total AS rate_per_100k, CRM3.total AS total_arrest,
               POP.population, SCH.school_rank
               FROM NJ_poverty AS POV 
               INNER JOIN NJ_crime AS CRM ON POV.county_name = CRM.county_name AND CRM.report_type = 'Number of Offenses'
               INNER JOIN NJ_crime AS CRM2 ON POV.county_name = CRM2.county_name AND CRM2.report_type = 'Rate Per 100,000'
               INNER JOIN NJ_crime AS CRM3 ON POV.county_name = CRM3.county_name AND CRM3.report_type = 'Number of Arrests'
               INNER JOIN (SELECT county_name, SUM(population) AS population FROM NJ_population GROUP BY county_name) AS POP ON POV.county_name = POP.county_name
               INNER JOIN (SELECT county_name, AVG(summativescore) AS school_rank FROM NJ_school_rating GROUP BY county_name) AS SCH ON POV.county_name = SCH.county_name'''
test = pd.read_sql_query(query, sqlite_connection)
test

In [ ]:
query = '''SELECT county_name, 'murder' AS crime_type, murder AS count FROM NJ_crime WHERE report_type = 'Number of Offenses'
        UNION ALL SELECT county_name, 'rape' AS crime_type, rape AS count FROM NJ_crime WHERE report_type = 'Number of Offenses'
        UNION ALL SELECT county_name, 'robbery' AS crime_type, robbery AS count FROM NJ_crime WHERE report_type = 'Number of Offenses'
        UNION ALL SELECT county_name, 'assault' AS crime_type, assault AS count FROM NJ_crime WHERE report_type = 'Number of Offenses'
        UNION ALL SELECT county_name, 'burglary' AS crime_type, burglary AS count FROM NJ_crime WHERE report_type = 'Number of Offenses'
        UNION ALL SELECT county_name, 'larceny' AS crime_type, larceny AS count FROM NJ_crime WHERE report_type = 'Number of Offenses'
        UNION ALL SELECT county_name, 'auto_theft' AS crime_type, auto_theft AS count FROM NJ_crime WHERE report_type = 'Number of Offenses'
        '''
test = pd.read_sql_query(query, sqlite_connection)
test

In [ ]:
query1 = '''SELECT DISTINCT "NJ-"||county_name AS id , county_name AS label,  '' AS parent, AVG(summativescore) AS value FROM 
        NJ_school_rating GROUP BY 1,2,3'''
df1 = pd.read_sql(query1, sqlite_connection)
df1.head()

In [ ]:
query2 = '''SELECT DISTINCT county_name||"-"||district_name AS id ,district_name AS label,  "NJ-"||county_name AS parent, AVG(summativescore) AS value FROM 
        NJ_school_rating GROUP BY 1,2,3'''
df2 = pd.read_sql(query2, sqlite_connection)
df2.head()

In [ ]:
query3 = '''SELECT DISTINCT district_name||"-"||school_name AS id ,school_name AS label,  county_name||"-"||district_name AS parent, AVG(summativescore) AS value FROM 
    NJ_school_rating GROUP BY 1,2,3'''
df3 = pd.read_sql(query3, sqlite_connection)
df3.head()

In [ ]:
query = '''SELECT county_name,district_name,gradespan,school_name,summativescore
            FROM 
              ( SELECT county_name,district_name,gradespan,school_name, summativescore,
                       ROW_NUMBER() OVER (PARTITION BY county_name
                                          ORDER BY summativescore DESC) AS rn
                FROM NJ_school_rating) AS tmp 
            WHERE rn <= 3
            ORDER BY county_name ; '''
test = pd.read_sql_query(query, sqlite_connection)
test

In [ ]:
test = test.to_dict(orient='records')
pprint(test)

In [ ]:
query1 = '''SELECT DISTINCT "NJ-"||county_name AS id , county_name AS label,  '' AS parent, SUM(population) AS value 
            FROM NJ_population                                                                                                  
            GROUP BY 1,2,3'''
df1 = pd.read_sql(query1, sqlite_connection)
df1.head()

In [ ]:
query2 = '''SELECT DISTINCT county_name||"-"||agency AS id ,agency AS label,  "NJ-"||county_name AS parent, SUM(population) AS value
            FROM NJ_population  GROUP BY 1,2,3'''
df2 = pd.read_sql(query2, sqlite_connection)
df2.head()

In [ ]:
sqlite_connection.close()

# JSON creation

In [ ]:
flare = {}
flare["name"] = "flare"
flare["description"] = "flare"
flare

In [ ]:
counties = list(test['county_name'].unique())
# counties

In [ ]:
children = []
for i in range(len(counties)):
    child1 = {}
    child1["name"] = counties[i]
    child1["description"] = test['rating'].loc[test['county_name']==counties[i]].mean()
    district = list(test['district_name'].loc[test['county_name']==counties[i]].unique())
    child2_list = []
    for k in range(len(district)):
        child2 = {}
        child2["name"] = district[k]
        child2["description"] = test['rating'].loc[(test['county_name']==counties[i]) & (test['district_name'] == district[k])].mean()
        child3_list = []
        gradespan = list(test['gradespan'].loc[(test['county_name']==counties[i]) & (test['district_name'] == district[k])].unique())
        for j in range(len(gradespan)):
            child3 = {}
            child3["name"] = gradespan[j]
            child3["description"] = test["rating"].loc[(test['county_name']==counties[i]) & (test['district_name'] == district[k]) & (test['gradespan'] == gradespan[j])].mean()
            child4_list = []
            for index,row in test.loc[(test['county_name']==counties[i]) & (test['district_name'] == district[k]) & (test['gradespan'] == gradespan[j])].iterrows():
                child4 = {}
                child4["name"] = row["school_name"]
                child4["description"] = row["rating"]
                child4["size"] = row["rating"]
                child4_list.append(child4)
            child3["children"] = child4_list
            child3_list.append(child3)
        child2["children"] = child3_list
        child2_list.append(child2)
    child1["children"] = child2_list
    children.append(child1)
children

In [ ]:
flare["children"] = children
flare

In [ ]:
with open("static/data/school.json", "w",encoding ='utf8') as outfile:  
    json.dump(flare, outfile, indent = 1) 

In [ ]:
metadata_df = pd.read_sql_query("SELECT T1.*, T2.population, T3.general_tax_rate, T3.effective_tax_rate FROM (SELECT county_name,median_hh_income,poverty_rate FROM NJ_poverty) AS T1 \
                            INNER JOIN (SELECT county_name, SUM(population) as population FROM NJ_population GROUP BY county_name) AS T2\
                            ON T1.county_name = T2.county_name \
                            INNER JOIN (SELECT county_name, AVG(general_tax_rate) as general_tax_rate, AVG(effective_tax_rate) as effective_tax_rate FROM NJ_tax GROUP BY county_name) AS T3 \
                            ON T1.county_name = T3.county_name", sqlite_connection)
metadata_dict = metadata_df.to_dict(orient='records')

school_df = pd.read_sql_query("SELECT county_name,district_code||school_code AS school_id, school_name,rating AS summativescore FROM NJ_school_rating WHERE county_name <> 'CHARTERS' ORDER BY county_name, rating DESC", sqlite_connection)
school_dict = school_df.to_dict(orient='records')

crime_df = pd.read_sql_query("SELECT county_name, murder,rape, robbery, assault, burglary, larceny, auto_theft, total FROM NJ_crime WHERE report_type = 'Number of Offenses'", sqlite_connection)
crime_dict = crime_df.to_dict(orient='records')

sqlite_connection.close()

data_json= {}
data_json["metadata"] = metadata_dict
data_json["school"] = school_dict
data_json["crime"] = crime_dict
data_json

In [ ]:
with open("static/data/plotly.json", "w",encoding ='utf8') as outfile:  
    json.dump(data_json, outfile, indent = 1) 

In [ ]:
query = '''SELECT county_name,district_name,general_tax_rate,effective_tax_rate FROM NJ_tax'''
test = pd.read_sql_query(query, sqlite_connection)
test

In [ ]:
flare = {}
flare["name"] = "flare"
flare["description"] = "flare"
flare

In [ ]:
counties = list(test['county_name'].unique())
# counties

In [ ]:
children = []
for i in range(len(counties)):
    child1 = {}
    child1["name"] = counties[i]
    child1["description"] = test['effective_tax_rate'].loc[test['county_name']==counties[i]].mean()
    district = list(test['district_name'].loc[test['county_name']==counties[i]].unique())
    child2_list = []
    for k in range(len(district)):
        for index,row in test.loc[(test['county_name']==counties[i]) & (test['district_name'] == district[k])].iterrows():
            child2 = {}
            child2["name"] = row["district_name"]
            child2["description"] = row["general_tax_rate"]
            child2["size"] = row["effective_tax_rate"]
            child2_list.append(child2)
    child1["children"] = child2_list
    children.append(child1)
children

In [ ]:
flare["children"] = children
flare

In [ ]:
with open("static/data/tax_sunburst.json", "w",encoding ='utf8') as outfile:  
    json.dump(flare, outfile, indent = 1) 

In [ ]:
query = ''' SELECT * FROM (SELECT county_name, police_dept, 'murder' AS crime_type, murder AS count FROM NJ_crime_detail  WHERE report_type = 'Rate Per 100,000' AND total <> 0
        UNION ALL SELECT county_name, police_dept, 'rape' AS crime_type, rape AS count FROM NJ_crime_detail  WHERE report_type = 'Rate Per 100,000' AND total <> 0
        UNION ALL SELECT county_name, police_dept, 'robbery' AS crime_type, robbery AS count FROM NJ_crime_detail  WHERE report_type = 'Rate Per 100,000' AND total <> 0
        UNION ALL SELECT county_name, police_dept, 'assault' AS crime_type, assault AS count FROM NJ_crime_detail  WHERE report_type = 'Rate Per 100,000' AND total <> 0
        UNION ALL SELECT county_name, police_dept, 'burglary' AS crime_type, burglary AS count FROM NJ_crime_detail  WHERE report_type = 'Rate Per 100,000' AND total <> 0
        UNION ALL SELECT county_name, police_dept, 'larceny' AS crime_type, larceny AS count FROM NJ_crime_detail  WHERE report_type = 'Rate Per 100,000' AND total <> 0
        UNION ALL SELECT county_name, police_dept, 'auto_theft' AS crime_type, auto_theft AS count FROM NJ_crime_detail  WHERE report_type = 'Rate Per 100,000' AND total <> 0)
        ORDER BY 1,2,3'''

In [ ]:
test = pd.read_sql_query(query, sqlite_connection)
test.head()

In [ ]:
flare = {}
flare["name"] = "crime"
flare["description"] = "crime"
flare

In [ ]:
counties = list(test['county_name'].unique())

In [ ]:
children = []
for i in range(len(counties)):
    child1 = {}
    child1["name"] = counties[i]
    child1["description"] = test['count'].loc[test['county_name']==counties[i]].mean()
    police_dept = list(test['police_dept'].loc[test['county_name']==counties[i]].unique())
    child2_list = []
    for k in range(len(police_dept)):
        child2 = {}
        child2["name"] = police_dept[k]
        child2["description"] = test['count'].loc[(test['county_name']==counties[i]) & (test['police_dept'] == police_dept[k])].mean()
        child3_list = []
        crime_type = list(test['crime_type'].loc[(test['county_name']==counties[i]) & (test['police_dept'] == police_dept[k])].unique())
        for j in range(len(crime_type)):
            for index,row in test.loc[(test['county_name']==counties[i]) & (test['police_dept'] == police_dept[k]) & (test['crime_type'] == crime_type[j])].iterrows():
                child3 = {}
                child3["name"] = row["crime_type"]
                child3["description"] = row["count"]
                child3["size"] = row["count"]
                child3_list.append(child3)
            child2["children"] = child3_list
        child2_list.append(child2)
    child1["children"] = child2_list
    children.append(child1)
children

In [ ]:
flare["children"] = children
flare

In [ ]:
with open("static/data/crime.json", "w",encoding ='utf8') as outfile:  
    json.dump(flare, outfile, indent = 1) 

In [ ]:
sqlite_connection.close()

# Linear regression

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sn
import scipy.stats as st
from scipy.stats import linregress
from scipy.stats import sem

In [ ]:
test.columns

In [ ]:
test.dtypes

In [ ]:
# Calculate the correlation coefficient and linear regression model 
x = test['tax'].astype(float)
# x = test['school_rank']
y = test['total_crime']

correlation = st.pearsonr(x,y)

# Print the answer to above calculation
print(f"""The correlation between school_rank and total_offense is {round(correlation[0],2)}.""")

# Calculate linear regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(x, y)
regress_values = x * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Plot linear regression on top of scatter plot
plt.scatter(x,y)
plt.plot(x,regress_values,"r-")

# Annotate linear regression
plt.annotate(line_eq,(20,36),fontsize=16,color="red")

# Add labels and title to plot
plt.xlabel("tax")
plt.ylabel("total_crime")
plt.title('total_offense by tax')
# plt.savefig("Images/scatter_line_plot.png")
plt.show()

# GeoJson creation

In [ ]:
geojson_url = 'https://opendata.arcgis.com/datasets/5f45e1ece6e14ef5866974a7b57d3b95_1.geojson'
geojson = requests.get(geojson_url).json()
# pprint(geojson)

In [ ]:
geojson.keys()

In [ ]:
geojson['features'][0].keys()

In [ ]:
geojson['features'][0]['geometry'].keys()

In [ ]:
geojson['features'][0]['properties']['COUNTY']

In [ ]:
#  SQLite DB creation and establishing connection
database_path = "Resources/NJ_CPS.sqlite"
engine = create_engine(f"sqlite:///{database_path}", echo=True)
sqlite_connection = engine.connect()

In [ ]:
query = '''SELECT T1.*, T2.school_rating, T3.tax_rate, T4.poverty_rate, T5.median_hh_income, T6.population
        FROM
        (SELECT county_name, total as crime_rate from NJ_crime WHERE report_type = 'Rate Per 100,000') AS T1
        INNER JOIN (SELECT county_name, AVG(rating) AS school_rating FROM NJ_school_rating  GROUP BY 1) AS T2 
        ON T1.county_name = T2.county_name
        INNER JOIN (SELECT county_name, AVG(effective_tax_rate) AS tax_rate FROM NJ_tax GROUP BY 1) AS T3
        ON T1.county_name = T3.county_name
        INNER JOIN (SELECT county_name, AVG(poverty_rate) AS poverty_rate FROM NJ_poverty GROUP BY 1) AS T4
        ON T1.county_name = T4.county_name
        INNER JOIN (SELECT county_name, AVG(median_hh_income) AS median_hh_income FROM NJ_poverty GROUP BY 1) AS T5
        ON T1.county_name = T5.county_name
        INNER JOIN (SELECT county_name, SUM(population) AS population FROM NJ_population GROUP BY 1) AS T6
        ON T1.county_name = T6.county_name
'''

In [ ]:
test = pd.read_sql_query(query, sqlite_connection)
test.head()

In [ ]:
for i in range(len(geojson['features'])):
    county = geojson['features'][i]['properties']['COUNTY']
    geojson['features'][i]['properties']['crime_rate'] = test['crime_rate'].loc[test['county_name']==county].item()
    geojson['features'][i]['properties']['school_rating'] = test['school_rating'].loc[test['county_name']==county].item()
    geojson['features'][i]['properties']['tax_rate'] = test['tax_rate'].loc[test['county_name']==county].item()
    geojson['features'][i]['properties']['poverty_rate'] = test['poverty_rate'].loc[test['county_name']==county].item()
    geojson['features'][i]['properties']['median_hh_income'] = test['median_hh_income'].loc[test['county_name']==county].item()
    geojson['features'][i]['properties']['population'] = test['crime_rate'].loc[test['county_name']==county].item()

In [ ]:
geojson['features'][0]

In [ ]:
with open("static/data/final.geojson", "w",encoding ='utf8') as outfile:  
    json.dump(geojson, outfile) 

In [ ]:
# test
f = open("static/data/final_data.json")
geojson = json.load(f)
pprint(geojson)

In [ ]:
connection_string = f"{user_nm}:{user_pw}@localhost:{user_port}/world_db"
engine = create_engine(f'postgresql://{connection_string}')